In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [9]:
original_df = pd.read_csv('../data/kc_house_data_original.csv')
original_df['sqft_basement'] = pd.to_numeric(original_df.sqft_basement.replace('?', None))

In [16]:
new_df = original_df
new_df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,basement,basement_ratio
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,0,0.000
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,1,0.156
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,0,0.000
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,1,0.464
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,0,0.000
5,7237550310,5/12/2014,1230000.0,4,4.50,5420,101930,1.0,0.0,0.0,3,11,3890,1530.0,2001,0.0,98053,47.6561,-122.005,4760,101930,1,0.282
6,1321400060,6/27/2014,257500.0,3,2.25,1715,6819,2.0,0.0,0.0,3,7,1715,1530.0,1995,0.0,98003,47.3097,-122.327,2238,6819,1,0.892
7,2008000270,1/15/2015,291850.0,3,1.50,1060,9711,1.0,0.0,NaN,3,7,1060,0.0,1963,0.0,98198,47.4095,-122.315,1650,9711,0,0.000
8,2414600126,4/15/2015,229500.0,3,1.00,1780,7470,1.0,0.0,0.0,3,7,1050,730.0,1960,0.0,98146,47.5123,-122.337,1780,8113,1,0.410
9,3793500160,3/12/2015,323000.0,3,2.50,1890,6560,2.0,0.0,0.0,3,7,1890,0.0,2003,0.0,98038,47.3684,-122.031,2390,7570,0,0.000


**How does presence of a basement/basement sq ft relate to housing price?**

- Basement vs. no basement: 3.63-152 (significant) 
- basement sqft: significant 
- basement sqft ratio: significant 

In [11]:
def basement_one_hot(x):
    x = int(x)
    if x == None: 
        return None 
    elif x > 0: 
        return 1
    else: 
        return 0

new_df['basement'] = new_df.sqft_basement.map(basement_one_hot)

#ttest comparing basement to no basement

with_basement = new_df[new_df.basement == 1].price.values
no_basement = new_df[new_df.basement == 0].price.values

t_stat, p_value = stats.ttest_ind(with_basement, no_basement)
print(f'P Value for Basement vs no Basement: {p_value}')

#use ols to determine p value for both basement sqft and basement ratio

new_df['basement_ratio'] = round(new_df.sqft_basement / new_df.sqft_living, 3)

#basement ratio 
ratio_model = ols(formula ='price~basement_ratio', data = new_df).fit()
ratio_model.summary()

#basement sqft 
basement_sqft_model =ols(formula = 'price~sqft_basement', data = new_df).fit() 
basement_sqft_model.summary()

P Value for Basement vs no Basement: 3.6343449691057825e-152


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     2451.
Date:                Sun, 05 Jul 2020   Prob (F-statistic):               0.00
Time:                        18:48:10   Log-Likelihood:            -3.0623e+05
No. Observations:               21597   AIC:                         6.125e+05
Df Residuals:                   21595   BIC:                         6.125e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      4.628e+05   2839.029    163.029      0.000    4.57e+05    4.68e+05
sqft_basement   265.2154      5.357     49.506      0.000     254.715     275.716
==============================================================================
Omnibus:                    17949.573   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           888026.296
Skew:                           3.697   Prob(JB):                         0.00
Kurtosis:                      33.531   Cond. No.                         635.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**How does sqft_living / sqft_living15 relate to housing price?**
- significant

In [5]:
new_df['sqft_living15_ratio'] = round(new_df.sqft_living / new_df.sqft_living15, 3)

ratio_living15_model = ols(formula ='price~sqft_living15_ratio', data = new_df).fit()

ratio_living15_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     2166.
Date:                Sun, 05 Jul 2020   Prob (F-statistic):               0.00
Time:                        15:36:15   Log-Likelihood:            -3.0636e+05
No. Observations:               21597   AIC:                         6.127e+05
Df Residuals:                   21595   BIC:                         6.127e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.756e+05   8190.131     21.438      0.000     1.6e+05    1.92e+05
sqft_living15_ratio  3.463e+05   7440.319     46.545      0.000    3.32e+05    3.61e+05
==============================================================================
Omnibus:                    18010.991   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           908163.223
Skew:                           3.710   Prob(JB):                         0.00
Kurtosis:                      33.889   Cond. No.                         6.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
ratio_model = ols(formula ='price~condition', data = new_df).fit()
ratio_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.11
Date:                Sun, 05 Jul 2020   Prob (F-statistic):           1.16e-07
Time:                        18:52:18   Log-Likelihood:            -3.0738e+05
No. Observations:               21597   AIC:                         6.148e+05
Df Residuals:                   21595   BIC:                         6.148e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4.709e+05   1.33e+04     35.322      0.000    4.45e+05    4.97e+05
condition   2.036e+04   3840.295      5.302      0.000    1.28e+04    2.79e+04
==============================================================================
Omnibus:                    19113.390   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1136802.126
Skew:                           4.025   Prob(JB):                         0.00
Kurtosis:                      37.619   Cond. No.                         20.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**How does sqft_lot / sqft_lot15 relate to housing price?**

In [6]:
new_df['sqft_lot15_ratio'] = round(new_df.sqft_lot / new_df.sqft_lot15, 3)

ratio_lot15_model = ols(formula ='price~sqft_lot15_ratio', data = new_df).fit()

ratio_lot15_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     37.79
Date:                Sun, 05 Jul 2020   Prob (F-statistic):           8.02e-10
Time:                        15:36:15   Log-Likelihood:            -3.0737e+05
No. Observations:               21597   AIC:                         6.147e+05
Df Residuals:                   21595   BIC:                         6.148e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         5.267e+05   3335.678    157.901      0.000     5.2e+05    5.33e+05
sqft_lot15_ratio  1.199e+04   1950.620      6.147      0.000    8167.618    1.58e+04
==============================================================================
Omnibus:                    19089.932   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1130404.734
Skew:                           4.018   Prob(JB):                         0.00
Kurtosis:                      37.519   Cond. No.                         2.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Are any columns not relevant to regression?**

In [3]:
#new_df.to_csv('../data/NEW.csv', index = False)
df = pd.read_csv('../data/NEW.csv')
df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,basement,basement_ratio,sqft_living15_ratio,sqft_lot15_ratio
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,0,0.000,0.881,1.000
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,1,0.156,1.521,0.948


In [4]:
# from tqdm import tqdm 
# def add_city_columns(df): 
#     url = 'http://www.mapquestapi.com/geocoding/v1/reverse?key=KEY&location={30.333472},-81.470448&includeRoadMetadata=true&includeNearestIntersection=true'

#     unique_county = [] 
#     for i in range(len(df)): 
#         row = df.iloc[i]
#         long = row.long
#         lat = row.lat 
#         print(long,lat)
#         break
     

# add_city_columns(df)

-122.257 47.5112


In [15]:
from tqdm import tqdm
def new_yr_renovated(df): 
    for i in tqdm(range(len(df))): 
        row = df.iloc[i]
        date = row.date 
        try:
            yr_renovated = int(row.yr_renovated)
            if yr_renovated == 0: 
                continue
            yr_renovated = int(yr_renovated)
            year = int(date.split('/')[-1])
            new_renovated = year - yr_renovated
            df.loc[i, 'yr_renovated'] = new_renovated
        except ValueError: 
            continue
    return df
new_df = new_yr_renovated(df)  

100%|██████████| 21597/21597 [00:05<00:00, 3764.73it/s]


In [17]:
new_df.to_csv('../data/NEW.csv')

In [7]:
assert False 

AssertionError: 

In [ ]:
unk_wf = df[df['waterfront'].isna()]
def estimate_waterfrontness(index):
    lat_range = (df.lat.max() - df.lat.min())
    long_range = (df.long.max() - df.long.min())
    chunk_size = 100
    lat_step = lat_range / chunk_size
    long_step = long_range / chunk_size
    return df[((df.loc[index, 'lat'] - lat_step) < df.lat) & (df.lat < (df.loc[index, 'lat'] + lat_step)) &
        ((df.loc[index, 'long'] - long_step) < df.long) & (df.long < (df.loc[index, 'long'] + long_step))].waterfront.mean()


for i in unk_wf.index:
    unk_wf.loc[i, 'waterfront'] = estimate_waterfrontness(i)